# World Markets Map
### Oscilação das bolsas mundias
Autor: Eduardo Teixeira / Scanner da Bolsa (https://scannerdabolsa.com.br/)


OBJETIVO: Criar um mapa mundi colorido e interativo, mostrando a variação das principais bolsas mundiais em tempo real

IMPORTANTE: É possível que o Github não execute o mapa ao rodar o código. Por isso recomendo que execute no Jupyter ou Colab



## 1. Bibliotecas utilizadas neste projeto

* Pandas: armazenamento e manipulação dos dados
* Datetime: data de consulta 
* Pandas Datareader: obter histórico dos ativos do Yahoo Finance
* Plotly: gerar a visualização gráfica

In [1]:
import pandas as pd

import datetime
from datetime import timedelta

from pandas_datareader import DataReader as data_reader

!pip install plotly
import plotly.express as px


## 2. Carregar a lista de países/índices

No csv adicionei algums dos principais países e seus respectivos índices de bolsa. Tentei adicionar o índice oficial de cada país, sempre que possível. Porém em alguns casos percebi que o Yahoo Finance não mantém histórico de diversos instrumentos, sendo um caso notável o índice do Reino Unido FTSE (^FTSE). Para estas situações fiz a substituição por outro índice semelhante, ou ainda algum ETF negociado em NYC, como os iShares da BlackRock. 

Um ponto que vale um bom debate é se seria mais interessante que todos os ativos da lista sejam ETFs negociados na bolsa de NY, assim "padronizando" sua conversão em USD e também com relação ao fuso horário, onde todos os ativos estariam sendo negociados no mesmo horário.

Você pode customizar esta lista conforme a sua necessidade, substituindo os tickers e também adicionando novos países. Para isso basta adicionar corretamente o código ISO de cada país, e o ticker (Symbol) do ativo desejado, do Yahoo Finance:

Lista de ISOs: https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes

Lista de ativos mundiais: https://finance.yahoo.com/world-indices



In [2]:
# carregando o csv em um dataframe e visualizando as primeiras linhas, para entender a estrutura da tabela

df = pd.read_csv('lista_paises.csv')
df.head()

,Country,ISO-code,Symbol
0,Brazil,BRA,^BVSP
1,Argentina,ARG,^MERV
2,Chile,CHL,ECH
3,Colombia,COL,ICOL
4,Peru,PER,EPU


## 3. Obter a cotação dos ativos

A função <b>DataReader</b> do pandas_datareader retorna o histórico de cotação do ativo através da fonte de dados escolhida (no caso o Yahoo Finance). Para obter o retorno (variação), é necessário efetuar o cálculo após a obtenção destes dados.

Neste código buscamos o histórico dos 10 últimos dias (timedelta 10), para garantir que pelo menos 2 cotações retornem e seja reaizado o cálculo da variação. Isto é devido a datas que caem em fim de semana ou longos períodos de feriados em que a bolsa pode ficar fechada.

Com um loop <b>for</b> fazemos o download de cada item da lista de países, e calculamos o retorno através da função <b>pct_change()</b> do pandas. Pegamos o registro mais atual desta coluna e adicionamos no dataframe principal, criando uma nova coluna chamada <b>Variação %</b>:

In [ ]:
for i, row in df.iterrows():
    
    print('processando',row['Country'],':',row['Symbol'])
    
    # dataframe temporario, para salvar o histórico de cotação do ativo e realizar o cálculo do retorno
    df_temp = data_reader(row['Symbol'], data_source='yahoo', start=str(datetime.date.today() - timedelta(10)))

    # calcular o retorno em numa nova coluna
    df_temp['Variação %'] = df_temp['Adj Close'].pct_change() * 100
    
    # salvar o valor de retorno mais atual no dataframe principal (df) na nova coluna criada (Variação %),
    # na linha respectiva do país (i)
    df.at[i,'Variação %'] = round(df_temp.iloc[-1]['Variação %'],2)  
    #df.at[i,'Variação %'] = df_temp.iloc[-1]['Variação %']
    
    # Avisar se houve algum erro na obtenção dos dados. Se sim, não será exibido no mapa
    if pd.isna(df.at[i,'Variação %']):
        print('---->   erro em',row['Symbol'],':', df.at[i,'Variação %'],'(Não será exibido no mapa)')
        

processando Brazil : ^BVSP
processando Argentina : ^MERV
processando Chile : ECH
processando Colombia : ICOL
processando Peru : EPU
processando Mexico : ^MXX
processando United States : ^GSPC
processando Canada : ^GSPTSE
processando United Kingdom : ISF.L
processando Germany : ^GDAXI
processando France : ^FCHI
processando Portugal : PGAL
processando Spain : ^IBEX
processando Italy : EWI
processando Switzerland : ^SSMI
processando Poland : ^PTL
processando Norway : ENOR
processando Sweden : ^OMX
processando Finland : ^OMXH25
processando Greece : GREK
processando Russia : IMOEX.ME
processando Turkey : XU100.IS
processando Egypt : EGPT
processando Nigeria : NGE
processando South Africa : JSE.JO
processando Saudi Arabia : KSA
processando UAE : UAE
processando Pakistan : PAK
processando India : ^NSEI
processando China : 000001.SS


### Dataframe atualizado com a nova coluna com os valores de variação de cada país:


In [ ]:
df.head(10)

## 4.Criar o Mapa Mundi (Plotly Cloropleth Map)

Um mapa coroplético representa uma superfície estatística por meio de áreas simbolizadas com cores, sombreamentos ou padrões de acordo com uma escala que representa a proporcionalidade da variável estatística em causa, como por exemplo a densidade populacional ou o rendimento per capita. Os símbolos criados a partir desta primitiva coincidem com as regiões onde foram coletados os dados, o que dá a impressão de que há uniformidade de dados dentro de cada uma das regiões e que as quebras ocorrem sempre nos limites destas áreas. (Wikipedia)

A biblioteca Plotly nos fornece uma forma muito prática de criar um <b>Cloropleth Map</b>, bastando informar a região (ISO-code) e o respectivo valor (Retorno), para que ele faça o preenchimento dos dados. Além disso, é possível customizar o elemento de diversas formas, conforme mostra o código abaixo:

In [ ]:
# definindo a fonte dos dados e coloração da escala
fig = px.choropleth(df, 
                    locations="ISO-code",    # identificação das regiões (países)
                    color="Variação %",      # coluna do df para popular os valores 
                    hover_name="Country",
                    range_color=[-5,5],      # limite de cores da escala. valores além deste numero terão a mesma cor do limite
                    color_continuous_scale = ["maroon","red","lightcoral","white","lightgreen","green","darkgreen"], 
                    color_continuous_midpoint=0)


# configurações de layout, título e fontes
fig.update_layout(
    width=1000,
    height=620,
    geo=dict(
        showframe=True,
        showcoastlines=True,
        landcolor = 'white',
        showocean=True, 
        oceancolor="azure",
        showlakes=True, 
        lakecolor="azure",
        projection_type='equirectangular'
    ),
    title={
        'text': 'Performance das Bolsas Mundiais',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    title_font_color='#525252',
    title_font_size=22,
        showlegend=False,
    font=dict(
        family='Sans-serif', 
        size=15, 
        color='#525252'
    ),
    annotations = [dict(
        x=0.5,
        y=0.25,
        xref='paper',
        yref='paper',
        text='<a href="https://scannerdabolsa.com.br/?utm_source=plots">@scannerdabolsa</a>',
        showarrow = False
    )]
)

# para mais customizações veja documentação: https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html


fig.show()